# Learn PySpark

![](https://miro.medium.com/v2/1*E27zNNFMwTvXj_XbO2RY-Q.png)

In [0]:
import requests
data = requests.get("https://public.tableau.com/app/sample-data/mobile_os_usage.csv")
dbutils.fs.put("/Volumes/workspace/default/volume/mobile_os_usage.csv", data.text)

In [0]:
%fs ls "/Volumes/workspace/default/volume"

In [0]:
%fs cp "/Volumes/workspace/default/volume/mobile_os_usage.csv" "/Volumes/workspace/default/volume/mobile_os_usage4.csv"

In [0]:
dbutils.widgets.combobox("db", "default", ["Database1","Database2","Database3","tbl1","tbl2","tbl3"])
dbutils.widgets.dropdown("tbl1", "mobile_os_usage", ["mobile_os_usage", "mobile_os_usage1", "mobile_os_usage2"])

In [0]:
dbutils.fs.cp("/Volumes/workspace/default/volume/mobile_os_usage1.csv", "/Volumes/workspace/default/volume/mobile_os_usage2.csv")

In [0]:
%sql
create table if not exists cities(id int,city string);
insert into cities values(3,'Mumbai'),(4,'Lucknow');
select * from cities;

In [0]:
%sql
from cities select *

In [0]:
%sql
show create table cities

In [0]:
df1=spark.read.csv("/Volumes/workspace/wd36_schema/ingestion_volume/source/cust")
df2=spark.read.csv("/Volumes/workspace/wd36_schema/ingestion_volume/source/cust_header", header=True)
df3=spark.read.csv("/Volumes/workspace/wd36_schema/ingestion_volume/source/cust_other_delim", sep="~", inferSchema=True).toDF("custid", "fname", "lname", "age", "prof")
df3=spark.read.csv("/Volumes/workspace/wd36_schema/ingestion_volume/source/cust_other_delim", sep="~", samplingRatio=0.0001)
#df1.createOrReplaceTempView("df1")
#spark.sql("select * from df1").show(2)
#display(df1)
#print(df3.printSchema())
#df3.show(6)
df3.where(df3.age > 40).show()

In [0]:
df3=spark.read.format("csv").option("sep", "~").option("inferSchema", "true").load("/Volumes/workspace/wd36_schema/ingestion_volume/source/cust_other_delim").toDF("custid", "fname", "lname", "age", "prof")
df3.show(6)

In [0]:
df3=spark.read.options(sep="~", inferSchema="true").csv("/Volumes/workspace/wd36_schema/ingestion_volume/source/cust_other_delim").toDF("custid", "fname", "lname", "age", "prof")
df3.show(6)

In [0]:
tbl_struct="id integer, fname string, lname string, age integer, prof string"
df3=spark.read.schema(tbl_struct).csv("/Volumes/workspace/wd36_schema/ingestion_volume/source/cust_other_delim", sep="~")
print(df3.printSchema())
df3.show(6)

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
custom_schema=StructType([
  StructField("custid", IntegerType(), False),
  StructField("fname", StringType(), True),
  StructField("lname", StringType(), True),
  StructField("age", IntegerType(), True),
  StructField("prof", StringType(), True)
])
df3=spark.read.schema(custom_schema).csv("/Volumes/workspace/wd36_schema/ingestion_volume/source/cust_other_delim", sep="~")
print(df3.printSchema())
df3.show(6)

In [0]:
#df3.write.json("/Volumes/workspace/default/volume/json")
df3.repartition(2).write.mode("overwrite").json("/Volumes/workspace/default/volume/json")

In [0]:
df3.repartition(2).write.mode("overwrite").orc("/Volumes/workspace/default/volume/orc")

In [0]:
#WORM - Wrinte Once Read Many
df3.write.mode("overwrite").parquet("/Volumes/workspace/default/volume/parquet")

In [0]:
#WMRM - Wrinte Many Read Many
df3.write.format("delta").mode("overwrite").save("/Volumes/workspace/default/volume/delta")

In [0]:
df3.write.mode("overwrite").saveAsTable("workspace.default.mobile_os_usage")
df3.write.mode("append").insertInto("workspace.default.mobile_os_usage")
display(spark.sql("select * from workspace.default.mobile_os_usage"))


In [0]:
%sh df -h /